In [ ]:
! pip install -q qiskit
! pip install -q qiskit-aer
! pip install -q qiskit-algorithms
! pip install -q qiskit-nature
! pip install -q qiskit-nature-pyscf # install the pyscf library
! pip install -q qiskit-ibm-runtime
# print(qiskit.__version__)
# print(qiskit_aer.__version__)
# print(qiskit_algorithms.__version__)
# print(qiskit_nature.__version__)
# print(qiskit_nature_pyscf.__version__)
# print(qiskit_ibm_runtime.__version__)

In [ ]:
import qiskit_nature
import qiskit
import qiskit_aer
import qiskit_algorithms
import qiskit_ibm_runtime
import qiskit_nature_pyscf
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.circuit.library import EfficientSU2
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_aer import  AerSimulator
# from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicBasis
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from scipy.linalg import eigh
from qiskit_nature.second_q.circuit.library import HartreeFock
from scipy.linalg import expm
from qiskit.quantum_info import Statevector
from numpy import pi

In [ ]:
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper

# Define hopping strength
t = -1.0
fermionic_hamiltonian = FermionicOp({
    "+_0 -_1": -t,
    "+_1 -_2": -t,
    "+_2 -_0": -t,
    "+_1 -_0": -t,
    "+_2 -_1": -t,
    "+_0 -_2": -t
    })

# Apply Jordan-Wigner transformation
mapper = JordanWignerMapper()
qubit_hamiltonian = mapper.map(fermionic_hamiltonian)
print("Qubit Hamiltonian (Pauli Operators):")
print(qubit_hamiltonian)

In [ ]:
from scipy.linalg import eigh
eign, eigv = eigh(qubit_hamiltonian.to_matrix())
print(eign)
print(eigv)

In [ ]:
from qiskit_aer import Aer
from qiskit_algorithms.optimizers import COBYLA
backend = Aer.get_backend('statevector_simulator')


steps = []
energy = []
class ParameterTracker:
    def __init__(self):
        self.params_history = []

    def callback(self, eval_count, params, mean, std):
        self.params_history.append((eval_count, params.copy(), mean))
        # print(f"Step {eval_count}: Energy = {mean:.6f}, Parameters = {params}")
        energy.append(mean)
        steps.append(eval_count)

tracker = ParameterTracker()


# Define the ansatz (EfficientSU2)
num_qubits = qubit_hamiltonian.num_qubits
ansatz = EfficientSU2(num_qubits, reps=1)
print(ansatz.decompose().draw(fold=-1))

# Define the optimizer
optimizer = COBYLA(maxiter=1000)

# Define the VQE instance
vqe = VQE(estimator=Estimator(), ansatz=ansatz, optimizer=optimizer, callback=tracker.callback)

# Run VQE
result = vqe.compute_minimum_eigenvalue(qubit_hamiltonian)

# Display results
print("\nFinal Results:")
print(f"Ground state energy: {result.eigenvalue.real}")
print("Parameters at each step:")
# for step in tracker.params_history:
#     step_idx, params, energy = step
#     print(f"Step {step_idx}: Energy = {energy:.6f}, Parameters = {params}")


# Plotting the energy at each step
plt.plot(steps, energy, marker='o', linestyle='-', color='b', linewidth=1.0, markersize=5)
# plt.title("VQE Optimization: Energy vs Steps")
plt.xlabel("Optimization Steps")
plt.ylabel("Energy")
plt.grid()
plt.show()

In [ ]:
# Backend for the simulation
from qiskit_aer import Aer
from qiskit_algorithms.optimizers import COBYLA
backend = Aer.get_backend('statevector_simulator')

# Custom callback to track parameters at each step
steps = []
energy = []
class ParameterTracker:
    def __init__(self):
        self.params_history = []

    def callback(self, eval_count, params, mean, std):
        self.params_history.append((eval_count, params.copy(), mean))
        # print(f"Step {eval_count}: Energy = {mean:.6f}, Parameters = {params}")
        energy.append(mean)
        steps.append(eval_count)

tracker = ParameterTracker()


# Define the ansatz (EfficientSU2)
num_qubits = qubit_hamiltonian.num_qubits
ansatz = EfficientSU2(num_qubits, reps=2, insert_barriers=True, entanglement="circular")
print(ansatz.decompose().draw(fold=-1))

# Define the optimizer
optimizer = COBYLA(maxiter=1000)

# Define the VQE instance
vqe = VQE(estimator=Estimator(), ansatz=ansatz, optimizer=optimizer, callback=tracker.callback)

# Run VQE
result = vqe.compute_minimum_eigenvalue(qubit_hamiltonian)

# Display results
print("\nFinal Results:")
print(f"Ground state energy: {result.eigenvalue.real}")
print("Parameters at each step:")
# for step in tracker.params_history:
#     step_idx, params, energy = step
#     print(f"Step {step_idx}: Energy = {energy:.6f}, Parameters = {params}")


# Plotting the energy at each step
plt.plot(steps, energy, marker='o', linestyle='-', color='b', linewidth=1.0, markersize=5)
# plt.title("VQE Optimization: Energy vs Steps")
plt.xlabel("Optimization Steps")
plt.ylabel("Energy")
plt.grid()
plt.show()

In [ ]:
# Backend for the simulation
from qiskit_aer import Aer
from qiskit_algorithms.optimizers import COBYLA
backend = Aer.get_backend('statevector_simulator')

# Custom callback to track parameters at each step
steps = []
energy = []
class ParameterTracker:
    def __init__(self):
        self.params_history = []

    def callback(self, eval_count, params, mean, std):
        self.params_history.append((eval_count, params.copy(), mean))
        # print(f"Step {eval_count}: Energy = {mean:.6f}, Parameters = {params}")
        energy.append(mean)
        steps.append(eval_count)

tracker = ParameterTracker()


# Define the ansatz (EfficientSU2)
num_qubits = qubit_hamiltonian.num_qubits
ansatz = EfficientSU2(num_qubits, reps=3, insert_barriers=True, entanglement="circular")
print(ansatz.decompose().draw(fold=-1))

# Define the optimizer
optimizer = COBYLA(maxiter=1000)

# Define the VQE instance
vqe = VQE(estimator=Estimator(), ansatz=ansatz, optimizer=optimizer, callback=tracker.callback)

# Run VQE
result = vqe.compute_minimum_eigenvalue(qubit_hamiltonian)

# Display results
print("\nFinal Results:")
print(f"Ground state energy: {result.eigenvalue.real}")
print("Parameters at each step:")
# for step in tracker.params_history:
#     step_idx, params, energy = step
#     print(f"Step {step_idx}: Energy = {energy:.6f}, Parameters = {params}")


# Plotting the energy at each step
plt.plot(steps, energy, marker='o', linestyle='-', color='b', linewidth=1.0, markersize=5)
# plt.title("VQE Optimization: Energy vs Steps")
plt.xlabel("Optimization Steps")
plt.ylabel("Energy")
plt.grid()
plt.show()

In [ ]:
# Backend for the simulation
from qiskit_aer import Aer
from qiskit_algorithms.optimizers import COBYLA
backend = Aer.get_backend('statevector_simulator')

# Custom callback to track parameters at each step
steps = []
energy = []
class ParameterTracker:
    def __init__(self):
        self.params_history = []

    def callback(self, eval_count, params, mean, std):
        self.params_history.append((eval_count, params.copy(), mean))
        # print(f"Step {eval_count}: Energy = {mean:.6f}, Parameters = {params}")
        energy.append(mean)
        steps.append(eval_count)

tracker = ParameterTracker()


# Define the ansatz (EfficientSU2)
num_qubits = qubit_hamiltonian.num_qubits
ansatz = EfficientSU2(num_qubits, reps=4, insert_barriers=True, entanglement="full")
print(ansatz.decompose().draw(fold=-1))

# Define the optimizer
optimizer = COBYLA(maxiter=1000)

# Define the VQE instance
vqe = VQE(estimator=Estimator(), ansatz=ansatz, optimizer=optimizer, callback=tracker.callback)

# Run VQE
result = vqe.compute_minimum_eigenvalue(qubit_hamiltonian)

# Display results
print("\nFinal Results:")
print(f"Ground state energy: {result.eigenvalue.real}")
print("Parameters at each step:")
# for step in tracker.params_history:
#     step_idx, params, energy = step
#     print(f"Step {step_idx}: Energy = {energy:.6f}, Parameters = {params}")


# Plotting the energy at each step
plt.plot(steps, energy, marker='o', linestyle='-', color='b', linewidth=1.0, markersize=5)
# plt.title("VQE Optimization: Energy vs Steps")
plt.xlabel("Optimization Steps")
plt.ylabel("Energy")
plt.grid()
plt.show()

In [ ]:
# Backend for the simulation
from qiskit_aer import Aer
from qiskit_algorithms.optimizers import COBYLA
backend = Aer.get_backend('statevector_simulator')

# Custom callback to track parameters at each step
steps = []
energy = []
class ParameterTracker:
    def __init__(self):
        self.params_history = []

    def callback(self, eval_count, params, mean, std):
        self.params_history.append((eval_count, params.copy(), mean))
        # print(f"Step {eval_count}: Energy = {mean:.6f}, Parameters = {params}")
        energy.append(mean)
        steps.append(eval_count)

tracker = ParameterTracker()


# Define the ansatz (EfficientSU2)
num_qubits = qubit_hamiltonian.num_qubits
ansatz = EfficientSU2(num_qubits, reps=5, insert_barriers=True, entanglement="full")
print(ansatz.decompose().draw(fold=-1))

# Define the optimizer
optimizer = COBYLA(maxiter=1000)

# Define the VQE instance
vqe = VQE(estimator=Estimator(), ansatz=ansatz, optimizer=optimizer, callback=tracker.callback)

# Run VQE
result = vqe.compute_minimum_eigenvalue(qubit_hamiltonian)

# Display results
print("\nFinal Results:")
print(f"Ground state energy: {result.eigenvalue.real}")
print("Parameters at each step:")
# for step in tracker.params_history:
#     step_idx, params, energy = step
#     print(f"Step {step_idx}: Energy = {energy:.6f}, Parameters = {params}")


# Plotting the energy at each step
plt.plot(steps, energy, marker='o', linestyle='-', color='b', linewidth=1.0, markersize=5)
# plt.title("VQE Optimization: Energy vs Steps")
plt.xlabel("Optimization Steps")
plt.ylabel("Energy")
plt.grid()
plt.show()

In [ ]:
# Backend for the simulation
from qiskit_aer import Aer
from qiskit_algorithms.optimizers import COBYLA, SPSA, GradientDescent, L_BFGS_B
backend = Aer.get_backend('statevector_simulator')

# Custom callback to track parameters at each step
steps = []
energy = []
class ParameterTracker:
    def __init__(self):
        self.params_history = []

    def callback(self, eval_count, params, mean, std):
        self.params_history.append((eval_count, params.copy(), mean))
        # print(f"Step {eval_count}: Energy = {mean:.6f}, Parameters = {params}")
        energy.append(mean)
        steps.append(eval_count)

tracker = ParameterTracker()


# Define the ansatz (EfficientSU2)
num_qubits = qubit_hamiltonian.num_qubits
ansatz = EfficientSU2(num_qubits, reps=5, insert_barriers=True, entanglement="sca", su2_gates=['ry', 'rz'])
print(ansatz.decompose().draw(fold=-1))

# Define the optimizer
optimizer = SPSA(maxiter=1000)

# Define the VQE instance
vqe = VQE(estimator=Estimator(), ansatz=ansatz, optimizer=optimizer, callback=tracker.callback)

# Run VQE
result = vqe.compute_minimum_eigenvalue(qubit_hamiltonian)

# Display results
print("\nFinal Results:")
print(f"Ground state energy: {result.eigenvalue.real}")
print("Parameters at each step:")
# for step in tracker.params_history:
#     step_idx, params, energy = step
#     print(f"Step {step_idx}: Energy = {energy:.6f}, Parameters = {params}")


# Plotting the energy at each step
plt.plot(steps, energy, marker='o', linestyle='-', color='b', linewidth=1.0, markersize=5)
# plt.title("VQE Optimization: Energy vs Steps")
plt.xlabel("Optimization Steps")
plt.ylabel("Energy")
plt.grid()
plt.show()